# Training PrimoGPT Model for NLP Feature Generation

This notebook implements the training process for the PrimoGPT model, which is designed to generate NLP features from stock market news and press releases. The model is fine-tuned using the Unsloth library for efficient training of large language models.

## Model Architecture
- Base Model: Meta-Llama-3.1-8B-Instruct
- Optimization: LoRA (Low-Rank Adaptation) for parameter-efficient fine-tuning
- Quantization: 4-bit quantization to reduce memory usage
- Context Length: 8192 tokens with RoPE scaling

## Training Configuration
- LoRA Parameters:
  - Rank (r): 16
  - Alpha: 16
  - Target modules: q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj
  - Dropout: 0 (optimized for performance)
  - Gradient checkpointing: "unsloth" for efficient long context training

- Training Parameters:
  - Batch size: 32
  - Learning rate: 2e-4
  - Epochs: 1
  - Optimizer: AdamW 8-bit
  - Weight decay: 0.01
  - LR scheduler: Linear

## Data Format
The training data follows the Alpaca format:
```
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}
```

## Model Saving
- Local saving for testing
- HuggingFace Hub upload for distribution

In [ ]:
# Install Unsloth library and dependencies (for Colab)

#%%capture
#!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

#from torch import __version__; from packaging.version import Version as V
#!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

## Import all necessary libraries and models

In [ ]:
from unsloth import FastLanguageModel
import torch

# Model configuration
max_seq_length = 8192
dtype = None
load_in_4bit = True

# List of 4bit pre-quantized models supported by Unsloth
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",
]

# Load the base model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

### We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
# Add LoRA adapters for efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,  
)

## Prepare data in Alpaca format

In [ ]:
import json
from datasets import Dataset

# Load the JSON data
with open('./data/merged_data.json', 'r') as f:
    data = json.load(f)

# Convert JSON data to Dataset format
dataset_dict = {
    "instruction": [],
    "input": [],
    "output": []
}

for item in data:
    dataset_dict["instruction"].append(item["instruction"])
    dataset_dict["input"].append(item["input"])
    dataset_dict["output"].append(item["response"])

# Create Dataset object
dataset = Dataset.from_dict(dataset_dict)

# Define Alpaca prompt format
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN to prevent infinite generation
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Apply formatting to dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

## Set up training arguments and trainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
# Display memory usage statistics
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

## Start training

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# Display final memory and time statistics
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Test the model

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "[SYSTEM PROMPT] You are a senior quantitative analyst specializing in stock market analysis. Your task is to analyze the provided company recent news and press releases to generate key features that could influence the stock's price movement in the next trading session. Focus on interpreting the given data to provide insights for algorithmic trading models. [COMPANY BASICS] Apple Inc is a company trading under the ticker AAPL on the NASDAQ NMS - GLOBAL MARKET. The company operates in the Technology industry with a market capitalization of $3,287,742. It has 161,000 employees. [RECENT NEWS] Here are the recent news articles related to AAPL: [Headline]: Apple Enables Touchless Airport Security Screening Experience [Summary]: Apple Inc (NASDAQ: AAPL) is working with several states, of which eight states will add their driver's license or state ID to Apple Wallet on their iPhone and Apple Watch for a touchless TSA airport security screening by replacing physical wallets. The Transportation Security Administration will enable select airport security checkpoints and lanes in participating airports as the first locations for driver's license or state ID in Wallet. Arizona and Georgia will be the first states to introduce [Headline]: Why Facebook Stock Dropped This Morning, but Apple and Roku Popped [Summary]: Shares of Facebook (NASDAQ: FB) stock slipped a modest 0.2% early this morning.  What's more interesting is why Facebook shares dipped in the first place -- and how this is connected to a 0.7% gain in Apple (NASDAQ: AAPL) stock and an even bigger 4.5% pop for Roku (NASDAQ: ROKU).  Turns out, Facebook suffered a downgrade this morning when analysts at Rosenblatt cut their rating on the shares from \"buy\" to just \"neutral.\" [Headline]: Apple Watch Series 7 could be delayed or hard to buy because of problems making it [Summary]: The upcoming new Apple Watch could be difficult to buy or delayed entirely, according to new reports.  The problems were reported by both Nikkei and Bloomberg, which said discussions between Apple and its suppliers were ongoing.  Apple is thought to be having trouble because of the complexities of a new design that is rumoured to be arriving with the Apple Watch Series 7. [Headline]: Apple to take deep dive into health space [Summary]: Rolfe Winkler, technology reporter for the Wall Street Journal, joins Fast Money to discuss Apple's deeper dive into health functions, including a tool to tell users when their blood pressure is on the rise. With Fast Money traders Tim Seymour and Nadine Terman. [Headline]: These are the three most important shows for Netflix: Analyst [Summary]: Nat Schindler, Bank of America Senior Internet, Digital Media Analyst joins Yahoo Finance's Alexandra Canal and the Yahoo Finance Live panel to discuss the latest news from Netflix. [Headline]: Google's Latest Foray Sparks More Bad News For Intel, AMD [Summary]: Alphabet Inc's (NASDAQ: GOOG) (NASDAQ: GOOGL) Google is developing its own central processors for notebooks and tablet computers, highlighting its Apple Inc (... [Headline]: Here Are The Top Stocks Cathie Wood Is Buying For Her Next Fund [Summary]: Cathie Wood plans a new exchange traded fund focused on transparency — and Apple and Microsoft make the cut. [Headline]: Google stock has quietly gone berserk — here's why [Summary]: The hottest FAANG stock around isn't Apple or Netflix, it's Google. Here's why. [Headline]: Why Fastly and Cloudflare Shares Jumped Today (and Akamai Ticked Higher) [Summary]: Shares of content delivery network (CDN) providers Fastly (NYSE: FSLY), Cloudflare (NYSE: NET), and Akamai Technologies (NASDAQ: AKAM) were all trading higher Wednesday.  Fastly and Cloudflare in particular had jumped sharply, up 7% and 6%, respectively, as of 1:10 p.m. EDT; Akamai was up about 1% as of this writing.  The tech stocks were likely up for two main reasons: a bullish day in the overall market (particularly for tech stocks) and a note from a Piper Sandler analyst saying a new security feature from Apple (NASDAQ: AAPL) could benefit CDN specialists Fastly, Cloudflare, and Akamai. [Headline]: Apple announces new digital drivers license partnership with 8 states [Summary]: Josh Lipton joins Closing Bell with news from Apple, which has partnered with eight states to roll out new digital drivers licenses that can be used in place of physical cards. [Headline]: Apple Asks All U.S. Employees to Report Vaccination Status [Summary]: (Bloomberg) -- Apple Inc. is asking all U.S. employees to report their vaccination status, marking the latest move in a Covid-19 campaign that has stopped short of mandating shots.The iPhone maker has asked the employees to report their status \"voluntarily\" by mid-September, regardless of whether they are working remotely or from an office. The company said it's using the data to inform its Covid-19 response efforts and protocols. Apple previously asked employees in California, Washington and Ne [Headline]: Google Might Design Its Own Computer Chips — Just Like Apple Did [Summary]: The tech giant may be planning to roll out computer processors made in-house in laptops and tablets from 2023, according to a report. [Headline]: Apple's stock peeked briefly into record territory after Wolfe Research boosted rating, price target [Summary]: Shares of Apple Inc. rallies briefly into record territory Wednesday, after Wolfe Research analyst Jeff Kvaal raised his rating, price target and earnings estimates, citing the belief that strong demand for the technology behemoth's iPhones will continue. [Headline]: Dow Jones Falls Amid Weak Jobs Data; Apple Stock Hits Record High; Wells Fargo Dives [Summary]: The Dow Jones fell even as Apple stock surged to a high. Wells Fargo stock flashed a sell signal. A cluster of stocks passed buy points. [Headline]: 10 Tech Stocks that Pay Dividends [Summary]: In this article, we will be taking a look at 10 tech stocks that pay dividends. To skip our detailed analysis of dividend investing, you can go directly to see the 5 Tech Stocks that Pay Dividends. The technology sector has been, and continues to be, one of the most fast-paced and innovative areas of […] [Headline]: Dow Jones Falls As Jobs Data Misses Estimates; Apple Hits All-Time High [Summary]: The Dow Jones Industrial Average reversed lower Wednesday, as jobs data missed estimates. Apple stock hit a new high. [Headline]: 3 Computing Stocks with Promise [Summary]: Computing stocks may be worth considering at this point. [Headline]: What Makes Apple (AAPL) a New Strong Buy Stock [Summary]: Apple (AAPL) might move higher on growing optimism about its earnings prospects, which is reflected by its upgrade to a Zacks Rank #1 (Strong Buy). [Headline]: Facebook's Latest Move To Win User Time From TikTok [Summary]: Facebook Inc (NASDAQ: FB) announced its Fantasy Games debut in the U.S. and Canada on the Facebook app for Apple Inc (NASDAQ: AAPL) iOS and Alphabet... [Headline]: Apple will let some apps guide users to websites to make payments, bypassing Apple's cut [Summary]: Apple on Wednesday said that some apps will be allowed to provide a link to their websites to prompt users to sign up for a subscription. [Headline]: Google Might Make Its Own Computer Chips — Just Like Apple Did [Summary]: The tech giant plans to roll out computer processors made in-house in laptops and tablets from 2023, according to a report. [LATEST PRESS RELEASE] Here is the most recent press release from AAPL (if available): [Headline]: Apple announces first states signed up to adopt driver's licenses and state IDs in Apple Wallet [Description]: Apple ® today announced that it is working with several states across the country, which will roll out the ability for their residents to seamlessly and securely add their driver' s license or state ID to Wallet on their iPhone ® and Apple Watch ®. Driver' s licenses and state IDs in Apple Wallet provide an easy, fast, and more secure way for people to present their... [Headline]: Japan Fair Trade Commission Closes App Store Investigation [Description]: Apple ® today announced an update coming to the App Store ® that closes an investigation by the Japan Fair Trade Commission. Before the change goes into effect in early 2022, Apple will update its guidelines and review process to make sure users of reader apps continue to have a safe experience on the App Store. The focus of the App Store is always to create a safe... [Headline]: Apple Changsha Opens Saturday in China [Description]: Apple ® today previewed Apple Changsha, the first Apple Store ® in Hunan province. Situated in the heart of the provincial capital city, Apple Changsha' s location provides easy access for customers across central China. Apple Changsha opens Saturday in the Changsha IFS mall, facing the bustling Huangxing Road. [ANALYSIS TASKS] Based primarily on the provided news and press releases (if available) for AAPL generate the following features defined in the output format. Remember your primary focus should be on interpreting the impact and implications of the most recent news and press releases on these features. [OUTPUT FORMAT] The output should be a markdown code snippet formatted in the following schema, including the leading and trailing \"```json\" and \"```\": ```json { \"news_relevance\": string  // How directly relevant the provided news and press releases are to the stock's performance (0: not relevant, 1: somewhat relevant, 2: highly relevant) \"sentiment\": string  // Overall sentiment towards the stock based on the news and press releases (-1: negative, 0: neutral, 1: positive) \"price_impact_potential\": string  // Potential impact of the news and press releases on the stock's price in the next trading session (0: no impact, 1: moderate impact, 2: high impact) \"trend_direction\": string  // Likely direction of the stock's price trend based on the news and press releases (-1: downward, 0: neutral, 1: upward) \"volume_impact\": string  // Potential impact on the stock's trading volume based on the news and press release relevance and sentiment (0: no impact, 1: moderate impact, 2: high impact) \"volatility_change\": string  // How the news and press releases might affect the stock's price volatility in the short term (0: no change, 1: moderate increase, 2: significant increase) } ``` IMPORTANT: Your response should ONLY include the JSON structure as specified above. Do not include any additional explanation or analysis.", # instruction
        "", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

## Save the model

In [ ]:
# Local saving
model.save_pretrained("PrimoGPT-Instruct") 
tokenizer.save_pretrained("PrimoGPT-Instruct")

# Online saving to HuggingFace
model.push_to_hub("One2Many/PrimoGPT-Instruct", token = "SECRET_TOKEN") 
tokenizer.push_to_hub("One2Many/PrimoGPT-Instruct", token = "SECRET_TOKEN") 